## Mixture Models Generally

Mixture models are generally defined as: 

$$p(x) = \sum_{k = 1}^{K}\pi_k p_k(x)$$

Where:

- $K$ is the number of groups/clusters
- $\pi_k$ is the weight showing how common group $K$ is ($0 \leq \pi_k \leq 1$ and $\sum_{k = 1}^K \pi_k  = 1$)
- $p_k(x)$ is the probability of $x$ for the k'th distribution

This equation tells us that our data are a combination of $K$ different groups with weights $\pi_k$ and probability distributions $p_k(x)$

In our lecture, we saw a simple example where our data was made up of 2 different normal (gaussian) distributions.

<img src="https://drive.google.com/uc?export=view&id=1DY1xFHTE2spjA7owrMy55jiANPOlKoXL" alt="Q"/>

This would have two groups ($K$ = 2). Each group is normally distributed ($p_1(x) = N(\mu_1, \Sigma_1), p_2(x) =  N(\mu_2, \Sigma_2)$) with equal weights ($\pi_1 = \pi_2 = 0.5$).



## Application to GMM

We just learned that mixture models say that data come from multiple ($K$) distributions ($p_k(x)$), each with different weights ($\pi_k$) based on how common each distribution is in the data.

Gaussian Mixture Modelling uses this idea to create clusters from our data. We pick $K$ (called `n_components` in sklearn), and ask sklearn to figure out

- what the clusters **are** (we assume they're multivariate normal, but ask sklearn to figure out the mean ($\mu_k$) and variance ($\Sigma_k$) of each one).
- the **probability** that each of our data points belongs to each cluster.


Officially, the algorithm we use is called the EM (Expectation-Maximization) algorithm, but it is pretty similar to how we found clusters in K-Means. 

Remember, in K-Means our steps were:

1. Randomly initiallize the centers of clusters
2. Assign every data point to it's closest cluster
3. Use that cluster assignment to move the center of the cluster
4. Repeat 2 and 3 until we have convergence. 

In GMM, we'll do the same thing BUT we have to account for the fact that 1) every data point belongs to every cluster and 2) we estimate the variance of each cluster instead of assuming its spherical.

1. Randomly initialize the centers (means) and variances of clusters
2. **E-Step**: Calculate the *responsibility* ($r_{nk}$) which tells us how likely a data point ($n$) is to be in a cluster ($k$)
3. **M-Step**: Use these *responsibilities* to update the mean and variance of each cluster.
4. Repeat 2 and 3 until we have convergence.

Looks pretty similar! Let's talk about the math for steps 2 and 3. 

## The E-Step

The E (or Expectation) Step to calculate the probability that a data point $n$ belongs to cluster $k$ for every data point, for every cluster. 

#### Question
If we have 100 data points ($n = 100$), and 5 clusters ($k = 5$), how many *responsibilites* will we be calculating?


FYI, We do this using the Normal/Gaussian density formula...

$N(\mu, \Sigma)=\left(\frac{1}{2\pi}\right)^{p/2}|\Sigma|^{-1/2}\exp\{-\frac{1}{2}(\textbf{x}-\mathbf{\mu})'\Sigma^{-1}(\textbf{x}-\mathbf{\mu})\}$

...but you don't need to memorize this density formula. Computers will do this part of the calculation for us.

### Responsibilities
The responsibilities ($r_{nk}$) tell us the probability that a data point $n$ belongs to cluster $k$. It is calculated as: 

$$ r_{nk} = \frac{\pi_k N(x_n |\mu_k, \Sigma_k)}{\sum_j \pi_j N(x_n | \mu_j, \Sigma_j)}$$

$\pi_k N(x_n | \mu_k, \Sigma_k)$ is the weight for cluster $k$ ($\pi_k$) times the probability of data point $x_n$ being from cluster $k$ (which we calculate using that multivariate normal density function above).

$$\underbrace{\pi_k}_\text{cluster $k$ weight} \underbrace{N(x_n | \mu_k, \Sigma_k)}_\text{probability of $x_n$ for cluster $k$}$$


It's important to include $\pi_k$ because if cluster $k$ has a lot of data points, than any given data point is pretty likely to be in cluster $k$. It's important to include $N(x_n | \mu_k, \Sigma_k)$ because it tells us whether a data point *is a good fit* with that cluster. 

## The M-Step
Once we've calculated all the *responsibilities*, we use them to update the means ($\mu_k$) and variances ($\Sigma_k$) of each of our clusters. 

Because every data point belongs to every cluster, every data point is used to estimate the mean and variance of each cluster. BUT not every data point is equally as influential. If a data point has a low chance of being in cluster 1, then it won't have much influence on the mean and variance of cluster 1. 

**Data points that have a high probability of being in a cluster have the most influence over its mean and variance** this is unlike K-Means where every data point in a cluster is equally influential.

### Updating Means

The formula for updating the mean of cluster $k$ is: 

$$\mu_k = \frac{1}{N_k}\sum_{n = 1}^N r_{nk}x_n$$

Compare that to the formula for calculating a regular mean (like in K-Means):

$$\mu_k = \frac{1}{N}\sum_{n = 1}^N x_n$$

You can see that each data point ($X_n$) is weighted by its responsibility $r_{nk}$. This is *why* data points with higher responsibilities have more influence.

The fraction at the begininning ($\frac{1}{N_k}$) is the sum of all the responsibilities for that cluster. Basically, it's a way of estimating how many data points are in a cluster when data points belong to more than one cluster ($N_k = \sum_{n = 1}^N r_{nk}$).


### Updating Variances
The formula for updating the variance of cluster $k$ is:
$$\Sigma_k = \frac{1}{N_k} \sum_{n = 1}^N r_{nk}(x_n-\mu_k)(x_n - \mu_k)^T$$

Compare this to the regular formula for calculating varinces:
$$\Sigma_k = \frac{1}{N} \sum_{n = 1}^N (x_n-\mu_k)(x_n - \mu_k)^T$$
Again, this looks like the typical formula for variance, except each data point is weighted by its responsibility.


### Updating $\pi_k$

The formula for updating the weights of each cluster is:

$$\pi_k = \frac{N_k}{N}$$

## A Simple, Univariate Example

data = 1,2,4,7,8,10

k = 2

### Step 1. Randomly Initialize Distributions

Let's assume:

Cluster 1: $p_1(x) = N(3,1)$, $\pi_1 = 0.5$

Cluster 2: $p_2(x) = N(6,1)$, $\pi_2 = 0.5$

### Step 2: E-Step

Now lets calculate the *responsibilities* of each data point. The function `sp.norm.pdf()` will calculate $N(\mu_k, \sigma_k)$ for us. 


|    | p(x \| mu = 3, var = 1) | p(x \| mu = 6, var = 1) |
|----|-------------------------|-------------------------|
| 1  | 0.05399                 | 0.00001                 |
| 2  | 0.24197                 | 0.00013                 |
| 4  | 0.24197                 | 0.05399                 |
| 7  | 0.00013                 | 0.24197                 |
| 8  | 0.00001                 | 0.05399                 |
| 10 | 0.00001                 | 0.00013                 |


In [ ]:
import scipy.stats as sp

data = [1,2,4,7,8,10]

# calculate the responsibilities

responsibilities = {"Cluster1": [],
"Cluster2":[]}

for d in data:
    # get the p_k(x) for both clusters
    p_k1 = #
    p_k2 = #
    # sp.norm.pdf(#, loc = #, scale = #)

    # calculate the responsibilities
    respon1 = # r_{n1}
    respon2 = # r_{n2}

    # store responsibilities in the dictionary
    responsibilities["Cluster1"].append(respon1)
    responsibilities["Cluster2"].append(respon2)

print(responsibilities)

### Step 3: M-Step

Now let's update $\mu_k$, $\Sigma_k$ and $\pi_k$.

In [ ]:
# means

## cluster 1 mean
sum1 = 0 # sum of data * responsibilities
nk1 = 0 # Nk1
sum2 = 0 # sum of data * responsibilities
nk2 = 0 # Nk2

for i in range(len(data)):
    # add the product of x and responsibility (cluster 1)
    

    #sum up the responsibilites for Nk1
    

    # add the product of x and responsibility (cluster 2)
    

    #sum up the responsibilites for Nk2
    

# calculate means
mu1 = #
mu2 = #
print("means    :", round(mu1,3), round(mu2,3))


# variances 
var1 = 0
var2 = 0

for i in range(len(data)):
    # add the product of the responsibility times (x-mu)**2
    

# calculate the varinces
var1 = #
var2 = #
print("variances:",round(var1,3),round(var2,3))

# pi_ks

pi1 = #
pi2 = #

print("weights  :",round(pi1,3),round(pi2,3))

## Step 4: Repeat

At this point, we would repeat the E and M steps using the new means, variances and $\pi_k$ that we printed above.


## Takeaways
Luckily, computers usually do the math for us, but seeing the math helps us understand *conceptually* what the algorithms are doing. 

The biggest takeaways from this lesson are:

- GMM does *soft assignment*, therefore (unlike K-Means) every data point has a probability of belonging to each cluster. Data points that are more likely to belong to a cluster have more *influence* on their means and variances. 
- GMM uses the EM algorithm to *iteratively* update the cluster distributions. It does this by first assinging a responsability to each data point (E-step), and then using them to calculate weighted means and variances for each cluster (M-step).
- Responsibilities ($r_{nk}$) measure the probability of a data point being in each cluster (technically it is the *posterior* probability).
- Responsibilities ($r_{nk}$) contain information about how common a cluster is ($\pi_k$), as well as the likelihood of a data point belonging to that cluster ($N(\mu_k, \Sigma_k$))